In [3]:
from tools import get_openapi_json, output_file, schemas2type, server2client
data_dict = get_openapi_json("http://127.0.0.1:8000/api/openapi.json")
schemas = schemas2type(data_schemas := data_dict['components']['schemas'])
api = server2client(data_paths := data_dict['paths'])
output_file('db_api.ts', api)
output_file('api_types.d.ts', schemas)

In [114]:
from tools.template import temp_header, temp_api


def server2client(data_paths: dict) -> str:
    api = ""
    for api_path, information in data_paths.items():
        # path name & parameters
        if len(api_path.split('{')) == 1:
            api_path = '"' + api_path + '"'
        else:
            text = ' + ' + \
                ' + '.join(api_path.split('{')[1:]).replace('}', ' ')
            api_path = '"' + api_path.split('{')[0] + '"' + text
            
        # mathod & all data
        for method, api_data in information.items():
            # fucntion name (use summary)
            api_name = (api_data['summary']).split()
            api_name = api_name[0].lower() + ''.join(api_name[1:])
            # parameters
            api_parameters = ','.join(
                [f'{parameter["name"]} : {parameter["schema"]["type"]}' for parameter in api_data['parameters']])
            api_parameters = api_parameters.replace(
                'array', '[]').replace('integer', 'number')
            # request body & parameters & path name (fucntion input...)
            api_body = ''
            if api_data.get('requestBody'):
                api_schemas = api_data['requestBody']['content']['application/json']['schema']['$ref'].split(
                    '/')[-1]
                api_body = '\n\t\tbody: JSON.stringify(data)' if len(
                    api_schemas) != 0 else ''
                api_parameters = (' ,' if len(api_parameters)
                                  else '') + api_parameters
                api_name += f'(data: {api_schemas} {api_parameters})'
            else:
                api_name += f'({api_parameters})'
            # responses type
            if api_data['responses']['200'].get('content'):
                api_responses = api_data['responses']['200']['content']['application/json']['schema']['$ref'].split(
                    '/')[-1]
                # for typescript
                api_name += f': Promise<{api_responses}>'
            # download file use apiFile fucntion (use path name to check)
            api += (temp_api % (api_name, api_path, method.upper(), api_body,
                    'api' if 'download' not in api_name else 'apiFile'))
    return temp_header+api
api = server2client(data_paths := data_dict['paths'])
api


'\nexport function getCookie(cname: string) {\n  let name = cname + "=";\n  let decodedCookie = decodeURIComponent(document.cookie);\n  let ca = decodedCookie.split(\';\');\n  for (let i = 0; i < ca.length; i++) {\n    let c = ca[i];\n    while (c.charAt(0) == \' \') {\n      c = c.substring(1);\n    }\n    if (c.indexOf(name) == 0) {\n      return c.substring(name.length, c.length);\n    }\n  }\n  return "";\n}\n\nexport async function api(url: string, options: RequestInit | undefined) {\n  // 取得 url 資源 和 選項\n  const BasisApiURL = "http://localhost:8000"\n  try {\n      // 請求\n      const res = await fetch(`${BasisApiURL}${url}`, options)\n      const data = await res.json()\n      return data\n  } catch (error) {\n      // 如果 請求失敗（Server 死咗 ）\n      return {\n          "error": "An error occurred.",\n          "code": 500,\n          "message": "Server Error."\n      }\n  }\n}\n\nexport async function apiFile(url: string, options: RequestInit | undefined) {\n  // 取得 url 資源 和 選項\n  co

In [47]:
from tools.template import temp_header, temp_api


def get_api_path(api_path: str) -> str:
    path_parts: list = ["'"]
    path_list: list = api_path.split('/')[1:]
    path_list_len: int = len(path_list)
    for index, path_item in enumerate(path_list):
        if path_item.startswith("{") and path_item.endswith("}"):
            path_item = f"' + {path_item[1:-1]}" + (" + '" if index + 1 < path_list_len else "")
            path_parts.append(path_item)
        else:
            path_parts.append(path_item if index + 1 < path_list_len else path_item + "'")
    return '/'.join(path_parts)


def get_api_parameters(parameters):
    return (','.join([f'{parameter["name"]} : {parameter["schema"]["type"]}' for parameter in parameters])).replace('array', '[]').replace('integer', 'number')


def server2client(data_paths: dict) -> str:
    api = ""
    for api_path, information in data_paths.items():
        # path name & parameters
        api_path = get_api_path(api_path)
        print(api_path)
    #     # mathod & all data
    #     for method, api_data in information.items():
    #         # fucntion name (use summary)
    #         summary: str = api_data['summary']
    #         api_name_lists: list[str] = summary.split()
    #         api_name = api_name_lists[0].lower(
    #         ) + ''.join([name.capitalize() for name in api_name_lists[1:]])
    #         # request body & parameters & path name (fucntion input...)
    #         api_body = ''
    #         api_parameters = get_api_parameters(api_data['parameters'])
    #         if api_data.get('requestBody'):
    #             api_schemas = api_data['requestBody']['content']['application/json']['schema']['$ref'].split(
    #                 '/')[-1]
    #             api_body = '\n\t\tbody: JSON.stringify(data)' if len(
    #                 api_schemas) != 0 else ''
    #             api_parameters = (' ,' if len(api_parameters)
    #                               else '') + api_parameters
    #             api_name += f'(data: {api_schemas} {api_parameters})'
    #         else:
    #             api_name += f'({api_parameters})'

    #         # responses type
    #         if api_data['responses']['200'].get('content'):
    #             api_responses = api_data['responses']['200']['content']['application/json']['schema']['$ref'].split(
    #                 '/')[-1]
    #             # for typescript
    #             api_name += f': Promise<{api_responses}>'
    #         # download file use apiFile fucntion (use path name to check)
    #         api += (temp_api % (api_name, api_path, method.upper(),
    #                 api_body, 'api' if 'download' not in api_name else 'apiFile'))
    # return temp_header+api
api = server2client(data_paths := data_dict['paths'])
api


'/api/token/pair'
'/api/token/refresh'
'/api/token/verify'
'/api/user'
'/api/user/' + uuid
'/api/user/' + uuid + '/password'
'/api/user/' + uuid + '/group'
'/api/group'
'/api/group/' + uuid
'/api/transaction'
'/api/transaction/' + uuid
'/api/transaction/download/csv'
'/api/todo'
'/api/todo/' + uuid
'/api/todo/' + uuid + '/done'
'/api/todo/download/csv'
